# Aries Basic Controller Example - Bob
## DID Exchange - Invitee

In this notebook we'll be responding to an invitation from Alice through our Agents. This notebook has the following phases:

<b>Begin with the Alice notebook ([localhost:8888]("http://localhost:8888"))</b>


6. Pull in dependencies
7. Instatiate the controller for our Agent
8. Set up a message listener running as a service on our controller
9. Paste the invitation from the Alice notebook into the invitation variable here
10. Accept the invitation

<b>Carry on at the Alice side</b>
    
14. Send Trust Ping to the Alice


### 6. Pull in dependencies


In [1]:
%autoawait
import time
import asyncio

IPython autoawait is `on`, and set to use `asyncio`


### 7. Instatiate the controller for our Agent

In [2]:
from aries_basic_controller.aries_controller import AriesAgentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8052
WEBHOOK_BASE = ""
ADMIN_URL = "http://bob-agent:8051"

agent_controller = AriesAgentController(webhook_host=WEBHOOK_HOST, webhook_port=WEBHOOK_PORT,
                                       webhook_base=WEBHOOK_BASE, admin_url=ADMIN_URL, connections=True)

### 8. Set up a listener for basicmessages events emitted by the controller as it receives webhooks

In [3]:
def messages_handler(payload):
    connection_id = payload["connection_id"]
    print("Handle messages ", payload, connection_id)

message_listener = {
    "handler": messages_handler,
    "topic": "basicmessages"
}

loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())

agent_controller.register_listeners([message_listener], defaults=True)

### 9. Paste the invitation from the Alice notebook into the invitation variable here;

In [5]:
#Paste in invitation from Alice agent
invite =  {'connection_id': 'a3792fc1-bf4a-4418-8a4d-88026714da25', 'invitation': {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation', '@id': '2d4d1711-8836-4b5b-a377-3254cea83cdd', 'serviceEndpoint': 'http://172.17.0.1:8020', 'label': 'Bob', 'recipientKeys': ['BhS8DZxNSCn3K2kfoKWZ2ttEsAvmNaJ5gAkmsB161kiR']}, 'invitation_url': 'http://172.17.0.1:8020?c_i=eyJAdHlwZSI6ICJkaWQ6c292OkJ6Q2JzTlloTXJqSGlxWkRUVUFTSGc7c3BlYy9jb25uZWN0aW9ucy8xLjAvaW52aXRhdGlvbiIsICJAaWQiOiAiMmQ0ZDE3MTEtODgzNi00YjViLWEzNzctMzI1NGNlYTgzY2RkIiwgInNlcnZpY2VFbmRwb2ludCI6ICJodHRwOi8vMTcyLjE3LjAuMTo4MDIwIiwgImxhYmVsIjogIkJvYiIsICJyZWNpcGllbnRLZXlzIjogWyJCaFM4RFp4TlNDbjNLMmtmb0tXWjJ0dEVzQXZtTmFKNWdBa21zQjE2MWtpUiJdfQ=='}

### 10. Accept the invitation, then move to Alice's notebook

In [6]:
# Receive Invitation
response = await agent_controller.connections.accept_connection(invite["invitation"])
# Print out accepted Invite and Alice's connection ID
print("Connection", response)
alice_id = response["connection_id"]


{'routing_state': 'none', 'their_label': 'Bob', 'connection_id': 'db9a91eb-5ad8-40ac-a4ed-c77e968953a2', 'updated_at': '2020-07-01 09:58:12.119219Z', 'created_at': '2020-07-01 09:58:12.096308Z', 'request_id': '568592fb-142b-4585-a4fe-76e0c0ec32d9', 'my_did': 'KeiroH5sJkLktn4xm7tHM7', 'accept': 'manual', 'invitation_mode': 'once', 'state': 'request', 'initiator': 'external', 'invitation_key': 'BhS8DZxNSCn3K2kfoKWZ2ttEsAvmNaJ5gAkmsB161kiR'}
Connection {'routing_state': 'none', 'their_label': 'Bob', 'connection_id': 'db9a91eb-5ad8-40ac-a4ed-c77e968953a2', 'updated_at': '2020-07-01 09:58:12.119219Z', 'created_at': '2020-07-01 09:58:12.096308Z', 'request_id': '568592fb-142b-4585-a4fe-76e0c0ec32d9', 'my_did': 'KeiroH5sJkLktn4xm7tHM7', 'accept': 'manual', 'invitation_mode': 'once', 'state': 'request', 'initiator': 'external', 'invitation_key': 'BhS8DZxNSCn3K2kfoKWZ2ttEsAvmNaJ5gAkmsB161kiR'}
Handle messages  {'connection_id': 'db9a91eb-5ad8-40ac-a4ed-c77e968953a2', 'message_id': 'ab96d723-eb9b

### 14. Send Trust Ping to the Alice

In [ ]:
# Do a trust ping to ensure connection is accepted by alice
trust_ping = await agent_controller.messaging.trust_ping(alice_id,"hello")
print("TUST PING TO ACTIVATE CONNECTION - Bob -> Alice")
print(trust_ping)   

#### 14.1 Check if connection state is active

In [ ]:
# Print connection list
connection = await agent_controller.connections.get_connection(researcher_id)
print("Alice AGENT CONNECTION")
print("State", connection["state"])

#### 14.2 Send a basic message over e2e DIDComm channel

In [ ]:
#send a basic message
message = await agent_controller.messaging.send_message(alice_id,"hello from bob world!")
print("BASIC MESSAGE - Bob -> Alice")
print(message)